[WordAndDoc2vec](https://github.com/darecophoenixx/wordroid.sblo.jp/wiki/WordAndDoc2vec)

In [ ]:
!pip install japanize-matplotlib

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import re
import itertools
import random

import numpy as np
import scipy
import pandas as pd
import gensim

from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import cluster, datasets, mixture
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import decomposition

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

from keras.utils import to_categorical

In [ ]:
import japanize_matplotlib

def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

plt.rcParams['font.family']

# load data

In [ ]:
df_part = pd.read_csv('../input/ks2018-encode-data-001/res_all_01.csv', index_col=0)
df_part.head()

# WordAndDoc2vec

In [ ]:
from feature_eng import lowcols

wd2v = lowcols.WD2vec(df_part)
wd2v

In [ ]:
# num_features = 12

# models = wd2v.make_model(num_features=num_features, embeddings_val=0.2, seed=10001)

# print('\n\n##################### model_gk1 >>>')
# models['model_gk1'].summary()
# print('\n\n##################### model_user >>>')
# models['model_user'].summary()
# print('\n\n##################### model >>>')
# model = models['model']
# model.summary()

In [ ]:
num_features = 12

from sklearn import decomposition
pca = decomposition.PCA(n_components=num_features)
pca.fit(df_part)
rscore = pca.transform(df_part) / 5
print(rscore.shape)

#cscore = pca.components_.T / 3
cscore = np.zeros((0,num_features))
for icol in df_part:
    v = df_part[icol]
    tmp = rscore[v==1].mean(axis=0, keepdims=True)
    cscore = np.r_[cscore, tmp]
print(cscore.shape)

models = wd2v.make_model(num_features=num_features, rscore=rscore, cscore=cscore)

print('\n\n##################### model_gk1 >>>')
models['model_gk1'].summary()
print('\n\n##################### model_user >>>')
models['model_user'].summary()
print('\n\n##################### model >>>')
model = models['model']
model.summary()

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

In [ ]:
wgt_lm = wd2v.get_wgt_bycol()
print(wgt_lm.shape)
df = pd.DataFrame(wgt_lm[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

### train

In [ ]:
%%time
hst, hst2 = wd2v.train(epochs=256, batch_size=32, verbose=0, lr0=0.001)
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst2.history["loss"]))), hst2.history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst2.history["loss"]))), hst2.history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst2.history["loss"]))), hst2.history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
%%time
hst = wd2v.train2(epochs=64, batch_size=32, verbose=0, lr0=0.001/8, flag_early_stopping=False)
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
wgt_prod = wd2v.get_wgt_bycol()
print(wgt_prod.shape)
df = pd.DataFrame(wgt_prod[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_upper(scatter)
g.map_lower(hexbin)

In [ ]:
ix, jx = 0,1
plt.figure(figsize=(10,10))
ax = sns.kdeplot(wgt_user[:,ix], wgt_user[:,jx],
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(wgt_prod[:,ix], wgt_prod[:,jx])
for ii, col in enumerate(df_part.columns):
    if ii in list(range(2,38)):
        text = col
        xy = (wgt_prod[ii,ix], wgt_prod[ii,jx])
        ax.annotate(text, xy=xy)

In [ ]:
ix, jx = 0,2
plt.figure(figsize=(10,10))
ax = sns.kdeplot(wgt_user[:,ix], wgt_user[:,jx],
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(wgt_prod[:,ix], wgt_prod[:,jx])
for ii, col in enumerate(df_part.columns):
    if ii in list(range(2,38)):
        text = col
        xy = (wgt_prod[ii,ix], wgt_prod[ii,jx])
        ax.annotate(text, xy=xy)

## 002

In [ ]:
wgt = np.r_[wgt_user, wgt_prod]
pca = decomposition.PCA(n_components=num_features)
pca.fit(wgt)
f = pca.transform(wgt) / 3
rscore = f[:wgt_user.shape[0]]
print(rscore.shape)
cscore = f[wgt_user.shape[0]:]

models = wd2v.make_model(num_features=num_features, rscore=rscore, cscore=cscore)

print('\n\n##################### model_gk1 >>>')
models['model_gk1'].summary()
print('\n\n##################### model_user >>>')
models['model_user'].summary()
print('\n\n##################### model >>>')
model = models['model']
model.summary()

In [ ]:
# rscore = wgt_user / 3
# print(rscore.shape)
# cscore = wgt_prod / 3

# models = wd2v.make_model(num_features=num_features, rscore=rscore, cscore=cscore)

# print('\n\n##################### model_gk1 >>>')
# models['model_gk1'].summary()
# print('\n\n##################### model_user >>>')
# models['model_user'].summary()
# print('\n\n##################### model >>>')
# model = models['model']
# model.summary()

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
# wgt_user = model.get_layer('user_embedding').get_weights()[0]
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

In [ ]:
wgt_lm = wd2v.get_wgt_bycol()
print(wgt_lm.shape)
df = pd.DataFrame(wgt_lm[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

In [ ]:
%%time
hst, hst2 = wd2v.train(epochs=128, batch_size=32, verbose=0, lr0=0.001)
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst2.history["loss"]))), hst2.history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst2.history["loss"]))), hst2.history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst2.history["loss"]))), hst2.history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
%%time
hst = wd2v.train2(epochs=64, batch_size=32, verbose=0, lr0=0.001/8, flag_early_stopping=False)
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('acc')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
wgt_prod = wd2v.get_wgt_bycol()
print(wgt_prod.shape)
df = pd.DataFrame(wgt_prod[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_lower(scatter)
g.map_upper(hexbin)

In [ ]:
wgt_user = wd2v.get_wgt_byrow()
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.set_context('paper')
g = sns.PairGrid(df, height=3.5)
g.map_diag(plt.hist, edgecolor="w")
g.map_upper(scatter)
g.map_lower(hexbin)

In [ ]:
ix, jx = 0,1
plt.figure(figsize=(10,10))
ax = sns.kdeplot(wgt_user[:,ix], wgt_user[:,jx],
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(wgt_prod[:,ix], wgt_prod[:,jx])
for ii, col in enumerate(df_part.columns):
    if ii in list(range(2,38)):
        text = col
        xy = (wgt_prod[ii,ix], wgt_prod[ii,jx])
        ax.annotate(text, xy=xy)

In [ ]:
ix, jx = 0,2
plt.figure(figsize=(10,10))
ax = sns.kdeplot(wgt_user[:,ix], wgt_user[:,jx],
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(wgt_prod[:,ix], wgt_prod[:,jx])
for ii, col in enumerate(df_part.columns):
    if ii in list(range(2,38)):
        text = col
        xy = (wgt_prod[ii,ix], wgt_prod[ii,jx])
        ax.annotate(text, xy=xy)

In [ ]:
ix, jx = 0,3
plt.figure(figsize=(10,10))
ax = sns.kdeplot(wgt_user[:,ix], wgt_user[:,jx],
                 cmap="Reds", shade=True, thresh=0.05)
ax.scatter(wgt_prod[:,ix], wgt_prod[:,jx])
for ii, col in enumerate(df_part.columns):
    if ii in list(range(2,38)):
        text = col
        xy = (wgt_prod[ii,ix], wgt_prod[ii,jx])
        ax.annotate(text, xy=xy)

In [ ]:
df_out_wgt_user = pd.DataFrame(wgt_user, index=df_part.index)
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.DataFrame(wgt_prod, index=df_part.columns)
df_out_wgt_col.head()

In [ ]:
df_out_wgt_user.to_csv('out_user.csv')
df_out_wgt_col.to_csv('out_col.csv')

In [ ]:
gamma = wd2v.get_gamma()
gamma, np.sqrt((1/gamma) / 2)

In [ ]:
np.save('out_gamma', gamma)